# 自動調參數
### 參數使用
```
class sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=1, iid=True, refit=True, cv=None, verbose=0, pre_dispatch=‘2*n_jobs’, error_score=’raise’, return_train_score=’warn’)

estimator：所使用的分類器，比如：estimator=RandomForestClassifier(min_samples_split=100, min_samples_leaf=20, max_depth=8, max_features=‘sqrt‘, random_state=10)，並且傳入除需要確定最佳的參數之外的其他參數。每個分類器都需要一個scoring參數或者score方法。
param_grid：值為字典或列表，即需要最優化的參數的取值，param_grid =param_test1，param_test1 = {‘n_estimators‘:range(10,71,10)}
scoring：準確評價標準，默認為None（使用estimator的誤差估計函數），這時需要使用score函數；或者如scoring=‘roc_auc‘，根據所選模型不同，評價準則不同。
cv：交叉驗證參數，默認為None
refit：默認為True，程序將會以交叉驗證訓練集得到的最佳參數，重新對所有可用的訓練集與測試集進行，作為最終用於性能評估的最佳模型參數。即在搜索參數結束後，用最佳參數結果再次fit一遍全部數據集。
iid:默認True,為True時，默認為各個樣本fold概率分布一致，誤差估計為所有樣本之和，而非各個fold的平均。

verbose：日誌冗長度，int：冗長度，0：不輸出訓練過程，1：偶爾輸出，>1：對每個子模型都輸出。

n_jobs: 並行數，int：個數,-1：跟CPU核數一致, 1:默認值。

pre_dispatch：指定總共分發的並行任務數。當n_jobs大於1時，數據將在每個運行點進行復制，這可能導致OOM，而設置pre_dispatch參數，則可以預先劃分總共的job數量，使數據最多被復制pre_dispatch次，進行預測的常用方法和屬性

grid.fit()：運行網格搜索

grid_scores_：給出不同參數情況下的評價結果

best_params_：描述了已取得最佳結果的參數的組合

best_score_：成員提供優化過程期間觀察到的最好的評分
```
### 屬性方法：

* grid.fit( train_x, train_y )：運行網格搜索
* grid_scores_：給出不同參數情況下的評價結果
* best_params_：描述了已取得最佳結果的參數的組合
* best_score_：成員提供優化過程期間觀察到的最好的評分
[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

# Progress bar for models fitting

> conda install keras-tqdm

[tqdm](https://github.com/bstriner/keras-tqdm)


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV#自動調參
from sklearn.feature_extraction import DictVectorizer
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#from keras_tqdm import TQDMCallback
from tqdm import tqdm

datapath = "data/titanic_data.csv"

def RandomForestTest():
    data = pd.read_csv(datapath)
    X = data[["Pclass","Sex","Age"]]
    Y = data["Survived"]

    X["Age"].fillna(X["Age"].mean(),inplace=True)  # 填空缺值

    X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)  

    DV = DictVectorizer(sparse=False)  # 轉成向量
    X_train = DV.fit_transform(X_train.to_dict(orient="records"))
    X_test = DV.transform(X_test.to_dict(orient="records"))
    
    # 實例化隨機森林分類器
    RF = RandomForestClassifier(verbose=0) #verbose=True #verbose=0, callbacks=[TQDMCallback()]
    
    # 測試超參數優化
    RF_param = {"n_estimators":[10,50,100,150,200],"max_depth":[5,10,15,20]}
    
    print(type(GridSearchCV(RF,param_grid=RF_param,cv=5)))
    GSCV = GridSearchCV(RF,param_grid=RF_param,cv=5)
    GSCV.fit(X_train,y_train)
    
    print("accuracy score",GSCV.score(X_test,y_test))
    print("The best model param :",GSCV.best_params_)



In [29]:
RandomForestTest()

<class 'sklearn.model_selection._search.GridSearchCV'>
accuracy score 0.8026905829596412
The best model param : {'max_depth': 5, 'n_estimators': 50}


In [64]:
pbar = tqdm(total = 100)
for i in range(100):
    pbar.set_description("Processing %d" %i)
    pbar.update(1)
pbar.close()



  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Processing 0:   0%|                                                                            | 0/100 [00:00<?, ?it/s]

Processing 1:   1%|▋                                                                  | 1/100 [00:00<00:00, 430.89it/s]

Processing 2:   2%|█▎                                                                 | 2/100 [00:00<00:00, 441.71it/s]

Processing 3:   3%|██                                                                 | 3/100 [00:00<00:00, 532.93it/s]

Processing 4:   4%|██▋                                                                | 4/100 [00:00<00:00, 557.20it/s]

Processing 5:   5%|███▎                                                               | 5/100 [00:00<00:00, 485.43it/s]

Processing 6:   6%|████                                                               | 6/100 [00:00<00:00, 530.76it/s]

Processing 7:   7%|████▋      

In [68]:
from time import sleep
from tqdm import tqdm, trange

for i in tqdm(range(1000)):
    sleep(0.01)



  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

  1%|▋                                                                                | 9/1000 [00:00<00:11, 83.18it/s]

  2%|█▌                                                                              | 19/1000 [00:00<00:11, 88.81it/s]

  3%|██▎                                                                             | 29/1000 [00:00<00:10, 90.79it/s]

  4%|███                                                                             | 39/1000 [00:00<00:10, 91.46it/s]

  5%|███▉                                                                            | 49/1000 [00:00<00:10, 91.64it/s]

  6%|████▋                                                                           | 59/1000 [00:00<00:10, 92.74it/s]

  7%|█████▌                                                                          | 69/1000 [00:00<00:10, 92.52it/s]

  8%|██████▎                  

In [89]:
times = 0
progress = tqdm(total=1000)

while times < 1001:
    progress.update(1)
    times += 1
    sleep(0.01)



100%|█████████████████████████████████████████████████████████████████████████████▉| 1000/1001 [00:30<00:00, 32.51it/s]


  1%|▊                                                                               | 10/1000 [00:00<00:10, 90.91it/s]

  2%|█▌                                                                              | 20/1000 [00:00<00:10, 93.28it/s]

  3%|██▍                                                                             | 30/1000 [00:00<00:10, 93.52it/s]

  4%|███▏                                                                            | 40/1000 [00:00<00:10, 93.18it/s]

  5%|████                                                                            | 50/1000 [00:00<00:10, 93.37it/s]

  6%|████▊                                                                           | 60/1000 [00:00<00:10, 93.55it/s]

  7%|█████▌                                                                          | 70/1000 [00:00<00:09, 93.53it/s]

  8%|██████▍                 